## Задание №1
Напишите функцию-представление handle, которая вернёт имя первого питомца в базе или ошибку 404, если питомцев нет.

In [ ]:
# Pet - модель с полем name
from django.http import HttpResponse
from django.http import Http404


def handle(request):
    pets_names = Pet.objects.values_list('name')
    if len(pets_names) == 0:
        raise Http404('No pets found :(')
    else:
        return HttpResponse(pets_names[0][0])

## Задание №2
Напишите функцию-представление handle, которая будет обрабатывать POST запрос. Оно должно проверять, есть ли пользователь с переданными данными в базе данных.

In [ ]:
from django.contrib.auth.models import User
from django.http import HttpResponse
import json


def handle(request):
    user_info = json.loads(request.body)
    names_query = list(User.objects.values('username'))
    emails_query = list(User.objects.values('email'))
    nam_list = [user['username'] for user in names_query]
    em_list = [email['email'] for email in emails_query]
    if user_info['username'] in nam_list and user_info['email'] in em_list:
        return HttpResponse(True)
    return HttpResponse(False)

# Пробовал через  authenticate, но всякий раз получал None :`(
# user = authenticate(username=user_info['username'], email=user_info['email'])
# if user:
#   return HttpResponse(True) ...

## Задание №3
Необходимо написать CBV PostView, основанное на классе, которое будет кодировать переданные строки в base64 и возвращать в формате json. Строки использовать в кодироаке utf-8.

In [ ]:
import json
from django.http import JsonResponse
from django.views.generic import View
import base64


# base64.b64encode(bytes('your string', 'utf-8'))

class PostView(View):

    def post(self, request):
        text = json.loads(request.body)
        for key, values in text.items():
            text[key] = [
                base64.b64encode(value.encode()).decode('utf-8')
                for value in values
            ]
        return JsonResponse(text)

## Задание №4
Напишите CBV PetListView для шаблона, который сформирует необходимый контекст для get запроса. Список pets в шаблоне должен состоять из 5 первых по алфавиту питомцев.

In [ ]:
# Pet - модель с полем name, template_path - строка
from django.views.generic.base import TemplateView


class PetListView(TemplateView):

    template_name = template_path

    def get_context_data(self):
        context = dict()
        context['pets'] = Pet.objects.order_by('name').all()[:5]
        return context

## Задание №5
Реализуйте функцию handle, которая сформирует необходимый контекст. Обработайте get и post запросы. При post запросе сделайте переход на url /auth, если форма валидная. При get запросе необходимо вывести поля формы.

In [ ]:
# Pet - модель с полем name, PetForm - форма для модели Pet с полем name
# template_path - строка, redirect_url - строка

from django.shortcuts import render, redirect


def handle(request):
    if request.method == 'POST':
        form = PetForm(request.POST, request.FILES)
        if form.is_valid():
            form.save()
            return redirect(redirect_url)
        return render(request, template_path, {'form': form})
    else:
        form = PetForm()
        return render(request, template_path, {'form': form})

## Задание №6
Реализуйте CBV PetView, которая сформирует необходимый контекст. Обработайте get и post запросы. При post запросе сделайте переход на url /auth, если форма валидная. При get запросе необходимо вывести поля формы.

